In [1]:
import sys
sys.path.append('/Users/mac/Downloads/RL-book-master')

In [2]:
from typing import Iterator, Mapping, Tuple, TypeVar, Sequence, List
from operator import itemgetter
import numpy as np

from rl.distribution import Distribution, Constant
from rl.function_approx import FunctionApprox
from rl.iterate import iterate
from rl.markov_process import (FiniteMarkovRewardProcess, MarkovRewardProcess,
                               RewardTransition)
from rl.markov_decision_process import (FiniteMarkovDecisionProcess, Policy,
                                        MarkovDecisionProcess,
                                        StateActionMapping)

S = TypeVar('S')
A = TypeVar('A')

In [3]:
def policy_iteration(
    mdp: MarkovDecisionProcess[S, A],
    gamma: float,
    approx_0: FunctionApprox[S],
    non_terminal_states_distribution: Distribution[S],
    num_state_samples: int
) -> Iterator[Tuple[FunctionApprox[S], Policy[S, A]]]:
    def greedy_policy_from_v(
        v: FunctionApprox[S],
    ) -> Policy[S, A]:
        def return_(s_r: Tuple[S, float]) -> float:
            s1, r = s_r
            return r + gamma * v.evaluate([s1]).item()
        class my_policy(Policy[S, A]):
            def act(self, s: S) -> Optional[Distribution[A]]:
                if mdp.is_terminal(s):
                    return None
                q_values = ((a, mdp.step(s, a).expectation(return_)) for a in mdp.actions(s))
                action = max(q_values, key=operator.itemgetter(1))[0]
                return Constant(action)
        return my_policy()
    def update(vf_policy: Tuple[FunctionApprox[S], Policy[S, A]])-> Tuple[FunctionApprox[S], Policy[S, A]]:

        vf, pi = vf_policy
        mrp: MarkovRewardProcess[S] = mdp.apply_policy(pi)
        policy_vf: FunctionApprox[S] = last(evaluate_mrp(mrp, gamma, vf, non_terminal, num_state))
        improved_pi: Policy[S, A] = greedy_policy_from_v(policy_vf)

        return policy_vf, improved_pi

    pi_0 = greedy_policy_from_v(approx_0)
    return iterate(update, (approx_0, pi_0))
    
    
    
    